<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/802%20code/t5-small-paradetox-1Token-split-masked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets
!pip install transformers[torch]
!pip install sacrebleu sentencepiece
!pip install evaluate
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration

raw_datasets = load_dataset("HamdanXI/paradetox-1Token-Split")
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint)

raw_datasets = raw_datasets.rename_column("en_toxic_comment", "input")
raw_datasets = raw_datasets.rename_column("en_neutral_comment", "labels")
raw_datasets = raw_datasets.remove_columns("edit_ops")
raw_datasets = raw_datasets.remove_columns("masked_comment")

In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'labels'],
        num_rows: 3784
    })
    validation: Dataset({
        features: ['input', 'labels'],
        num_rows: 811
    })
    test: Dataset({
        features: ['input', 'labels'],
        num_rows: 811
    })
})

In [10]:
max_token_length_input = max(len(tokenizer.encode(item)) for item in raw_datasets["train"]["input"])
max_token_length_label = max(len(tokenizer.encode(item)) for item in raw_datasets["train"]["labels"])

print(f"Max token length for input: {max_token_length_input}")
print(f"Max token length for label: {max_token_length_label}")

if max_token_length_input > max_token_length_label:
    highest_length = max_token_length_input
else:
    highest_length = max_token_length_label

print(highest_length)

Max token length for input: 42
Max token length for label: 41
42


In [13]:
def tokenize_function(example):
    source = example["input"]
    target = example["labels"]

    # Tokenizing source and target without returning tensors and without padding
    tokenized_source = tokenizer(source, truncation=True, max_length=highest_length, return_tensors="pt")
    tokenized_target = tokenizer(target, truncation=True, max_length=highest_length, return_tensors="pt")

    return {
        "input_ids": tokenized_source["input_ids"][0],
        "attention_mask": tokenized_source["attention_mask"][0],
        "labels": tokenized_target["input_ids"][0]
    }

tokenized_datasets = raw_datasets.map(tokenize_function)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.1)
# tokenized_datasets["validation"] = tokenized_datasets.pop("test")

Map:   0%|          | 0/3784 [00:00<?, ? examples/s]

In [16]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(output_dir="t5-small-paradetox-1Token-split-masked", push_to_hub=True)
training_args = training_args.set_save(steps=10000)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [17]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.552600
1000,0.365300


TrainOutput(global_step=1419, training_loss=0.42003064360561465, metrics={'train_runtime': 571.3626, 'train_samples_per_second': 19.868, 'train_steps_per_second': 2.484, 'total_flos': 76895290392576.0, 'train_loss': 0.42003064360561465, 'epoch': 3.0})

In [18]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

events.out.tfevents.1700564778.fa8cbad48d94.5548.0:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

'https://huggingface.co/HamdanXI/t5-small-paradetox-1Token-split-masked/tree/main/'